In [6]:
var pos = require('pos');
var words = new pos.Lexer().lex('This is some sample text. This text can contain mucho suerte.');
var tagger = new pos.Tagger();
tagger.extendLexicon({'mucho': ['JJ', 'RB']});
var taggedWords = tagger.tag(words);
for (var i in taggedWords) {
    var taggedWord = taggedWords[i];
    var word = taggedWord[0];
    var tag = taggedWord[1];
    console.log(word + " /" + tag);
}

// // extend the lexicon
// tagger.extendLexicon({'Obama': ['NNP']});
// tagger.tag(['Mr', 'Obama']);

This /DT
is /VBZ
some /DT
sample /NN
text /NN
. /.
This /DT
text /NN
can /MD
contain /VB
mucho /JJ
suerte /NN
. /.


undefined

In [5]:
var util = require('util');

var myLog = (obj) => console.log(util.inspect(obj, false, null));

'use strict'

In [6]:
var ParseEnglish = require('parse-english'),
    english = new ParseEnglish();

/**
 * parse-latin would fail helplessly at the full-stop preceding the
 * capital `H`, and would erroneously parse the following as two
 * sentences.
 */
myLog(english.parse(
  'A hapless but friendly City of London worker is here.'
));

{ type: 'RootNode',
  children: 
   [ { type: 'ParagraphNode',
       children: 
        [ { type: 'SentenceNode',
            children: 
             [ { type: 'WordNode',
                 children: 
                  [ { type: 'TextNode',
                      value: 'A',
                      position: 
                       Position {
                         start: { line: 1, column: 1, offset: 0 },
                         end: { line: 1, column: 2, offset: 1 } } } ],
                 position: 
                  Position {
                    start: { line: 1, column: 1, offset: 0 },
                    end: { line: 1, column: 2, offset: 1 } } },
               { type: 'WhiteSpaceNode',
                 value: ' ',
                 position: 
                  Position {
                    start: { line: 1, column: 2, offset: 1 },
                    end: { line: 1, column: 3, offset: 2 } } },
               { type: 'WordNode',
                 children: 
                  [ {

undefined